In [10]:
import numpy as np
import cv2 as cv
#img = cv.imread('home.jpg')

img = cv.imread("C:\\Users\\rain\\Pictures\\B0006674.jpg")
img = cv.resize(img, (400, 400), interpolation=cv.INTER_AREA)
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

sift = cv.SIFT_create()
kp = sift.detect(gray,None)

img = cv.drawKeypoints(gray,kp,img,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv.imwrite('sift_keypoints.jpg',img)

True

In [2]:
def calc_dist(vecA, vecB):
    return np.sqrt(np.sum(np.power(vecA - VecB, 2)))
def k_means(data, k):
    # Initialization of k mean vectors
    cluster_belong = np.zeros(data.shape[0],int)
    mean_vectors = np.zeros_like(data)
    
    for i in range(k):
        rn = random.randint(0,data.shape[0])
        mean_vectors[k] = data[rn,:].copy()
    #
    go_on = True
    while(go_on):
        go_on = False
        
        #Assign each point to one cluster
        for i in range(data.shape[0]):
            last_cluster_belong = cluster_belong[i]
            min_dist = calc_dist(mean_vectors[0], data[i])
            cluster_belong[i] = 0
            for j in range(1, k):
                if calc_dist(mean_vectors[j], data[i]) < min_dist:
                    min_dist = calc_dist(mean_vectors[j], data[i])
                    cluster_belong[i] = j
            # if the cluster of any point changes, continue with the next iteration
            if last_cluster_belong != cluster_belong[i]:
                go_on = True
        # Update the mean vectors of newly generated clusters
        for i in range(k):
            a = np.where(cluster_belong == i)
            total = np.zeros(data.shape[1], )
            average = np.zeros(data.shape[1], )
            for j, index in enumerate(a[0]):
                total += data[index]
                average = (total/(j+1))
            mean_vectors[i] = average.copy()
    return [mean_vectors, cluster_belong]

In [3]:
def bags_of_word(images)
    img_features = SIFT(images) # [img_num, feature_num, feature_dim]
    img_features_resize = np.resize(img_features,\
                                   (img_features.shape[0]*img_features.shape[1], img_features.shape[-1]))
    mean_vectors, cluster_belong = k_means(img_features, k)
    cluster_belong = np.resize(cluster_belong,\
                              (img_features.shape[0], img_features.shape[1]))
    histograms = np.zeros(img_features.shape[0], int)
    for idx, img inenumerate(cluster_belong):
        statistic = Counter(img) # represent the image with a histogram over all clusters
        for cluster in statistic:
            histograms[idx,clluster] = statistic[cluster]
    return histograms

SyntaxError: invalid syntax (3548017481.py, line 1)

In [11]:
import os
import cv2
import numpy as np
import glob
from sklearn import svm
import joblib
import scipy.io
WORD_COUNT = 50

# compute SIFT Features of an image using opencv
def calcSiftFeature(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    # 计算图片的特点和特征点描述
    keypoints, features = sift.detectAndCompute(img, None)
    return features

# 计算词袋
def learnVocabulary(features):
    # criteria表示迭代停止的模式 eps-精度0.1， max_iter-满足最大迭代次数20
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.1)
    # 得到k-means聚类的初始中心点
    flags = cv2.KEAMEANS_RANDOM_CENTERS
    # 标签，中心 = keameans(输入数据（特征）、聚类的个数k，预设标签，聚类停止条件，重复聚类次数，初始聚类中心点)
    compactness, labels, centers = cv2.kmeans(features, WORD_COUNT, None, criteria, 20, flags)
    return centers

# 计算特征向量
def calcFeatVec(features, centers):
    featVec = np.zeros((1, WORD_COUNT))
    for i in range(0, features.shape[0]):
        # 第i张图片的特征点
        fi = features[i]
        diffMat = np.tile(fi, (WORD_COUNT, 1)) - centers
        # axis=1按行求和，即特征到每个中心点的距离
        sqSum = (diffMat**2).sum(axis=1)
        dist = sqSum**0.5
        # 升序排序
        sortedIndices = dist.argsort()
        # 取出最小的距离，即找到最近的中心点
        idx = sortedIndices[0]
        # 该中心点对应+1
        featVec[0][idx] += 1
    return featVec

# 训练svm分类器
def SVM_Train(data_vec, labels):
    # 设置SVM模型参数
    clf = svm.SVC(decision_function_shape='ovo')
    # 利用x_train,y_train训练SVM分类器，获得参数
    clf.fit(data_vec,labels)
    return clf

# SVM分类器测试集正确率
def SVM_Test(svm_model,centers,images_dir,features_dir):
    # 计算每张图片的特征向量
    data_vec,labels = cal_vec(images_dir, centers, features_dir)
    res = svm_model.predict(data_vec)
    num_test = data_vec.shape[0]
    acc = 0
    for i in range(num_test):
        if labels[i] == res[i]:
            acc = acc + 1
    return acc/num_test,res,labels